# TOOLS

1. FunctionTool: Convert any python function into a tool that the agent can use.

2. QueryEngineTool: A tool that lets agents use query engines.

3. ToolSpecs: Sets of tools created byt the community, which often include tools for specific services like Gmail.

4. Utility Tools: Special tools that help handle large amounts of data from other tools.


In [2]:
from llama_index.core.tools import FunctionTool
def get_weather(location:str)->str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny."
tool=FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="useful for getting the weather for a given location.",
)
tool.call("New York")

Getting weather for New York


ToolOutput(content='The weather in New York is sunny.', tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny.', is_error=False)

In [3]:
'''Query Engine tool'''
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
embed_model=HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")
db=chromadb.PersistentClient(path='./alfred_chroma_db')
chroma_collection=db.get_or_create_collection("alfred")
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)
index=VectorStoreIndex.from_vector_store(vector_store,embed_model=embed_model)
llm= HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine=index.as_query_engine(llm=llm)
tool=QueryEngineTool.from_defaults(query_engine,name="query",description='put up a query and get your answer')

# Toolspecs

- collections of tools that work together like a well organised professional toolkit.


In [4]:
from llama_index.tools.google import GmailToolSpec
tool_spec=GmailToolSpec()
tool_spec_list=tool_spec.to_tool_list()

In [6]:
[(tool.metadata.name,tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\n\n        Searches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query (str): The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\n\n        Create and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n     

# MCP in LlamaIndex

- model context protocol


In [9]:
from llama_index.tools.mcp import BasicMCPClient,McpToolSpec
mcp_client=BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool=McpToolSpec(client=mcp_client)
agent=await get_agent(mcp_tool)
agent_context=Context(agent)

NameError: name 'get_agent' is not defined

# Utility Tools

1. OnDemandToolLoader: This tool turns any existing LlamaIndex data loader into a tool that an agent can use. The tool can be called with all the parameters needed to trigger load_data from the data loader, along with a natural language query string. During execution, we first load data from the data loader, index it and then query it on demand.
2. LoadAndSearchToolSpec: It takes in any existing Tool as input. As a tool spec, it implements to_tool_list and when that function is called two tools are returned: a loading tool and then a search tool. The load tool execution would call the underlying Tool, and then index the ouptut. The search tool execution would take in a query string and call the underlying index.
